# Task 4.7 – Aggregating, Deriving and Exporting Data

In this task, I will:
- Derive new variables from existing ones
- Label price ranges and identify busiest order times
- Create grouped categories for days and hours
- Export the enhanced dataframe for future analysis

## Import Libraries and Set Path

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Define base path
path = r'/Users/canancengel/A4_Instacart Basket Analysis/02_Data'

## Step 1: Import the merged dataset

In this step, I load the fully merged orders-products dataframe (`ords_prods_merge.pkl`) that was created in the previous exercise. This file contains all the necessary information for further analysis—every order, its products, and related product metadata are now combined into a single dataset.

Checking the shape confirms that the merged dataframe includes **32,404,859 rows** and **15 columns**, ensuring that no data was lost in previous steps.

We will use this complete dataset as the basis for all new variable derivations and aggregations in this task.

In [3]:
# Load merged dataset
df = pd.read_pickle(os.path.join(path, 'Prepared Data', 'ords_prods_merge.pkl'))

In [4]:
# Check shape
df.shape

(32404859, 15)

## Step 2: Create a new column for price ranges

I'll create the `price_label` column based on the value of the `prices` column.
- Low-range product: price ≤ 5
- Mid-range product: 5 < price ≤ 15
- High-range product: price > 15

In [5]:
df['prices'].describe()

count    3.240486e+07
mean     1.198023e+01
std      4.956554e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      9.999900e+04
Name: prices, dtype: float64

In [6]:
df[df['prices'] > 15]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices
398,2086598,6,1,5,18,NaN,True,38293,2,0,both,Ground Turkey Breast,35,12,19.6
401,298250,6,2,4,16,6.0,False,38293,1,1,both,Ground Turkey Breast,35,12,19.6
436,2402008,7,2,1,19,30.0,False,13198,14,0,both,85% Lean Ground Beef,122,12,23.2
465,121053,7,3,0,18,30.0,False,13198,22,1,both,85% Lean Ground Beef,122,12,23.2
480,1695742,7,4,2,10,9.0,False,13198,13,1,both,85% Lean Ground Beef,122,12,23.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404506,2017995,206208,34,6,15,7.0,False,13877,5,1,both,Smoked Salmon,15,12,21.6
32404588,3356245,206208,40,5,9,10.0,False,13877,11,1,both,Smoked Salmon,15,12,21.6
32404601,3356245,206208,40,5,9,10.0,False,9339,24,1,both,Organic Chicken & Apple Sausage,106,12,21.1
32404672,3292671,206208,45,2,11,3.0,False,7952,16,1,both,Chicken & Apple Breakfast Sausage,106,12,19.3


In [7]:
df['prices'].isnull().sum()

0

In [8]:
df[df['prices'].isnull()]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices


In [9]:
# Create price range labels
df['price_label'] = df['prices'].apply(
    lambda x: 'Low-range product' if x <= 5 
    else ('Mid-range product' if x > 5 and x <= 15 else 'High-range product')
)

In [10]:
# View distribution
df['price_label'].value_counts(dropna=False)

price_label
Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: count, dtype: int64

In [11]:
def price_label(row):
    if row['prices'] <= 5:
        return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid-range product'
    elif row['prices'] > 15:
        return 'High-range product'
    else:
        return 'Not enough data'

df['price_range'] = df.apply(price_label, axis=1)
print(df.columns)

Index(['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'new_customer', 'product_id',
       'add_to_cart_order', 'reordered', '_merge', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_label', 'price_range'],
      dtype='object')


## Step 3: Create a column for busiest days of the week

Determine the two busiest and two slowest days in terms of total orders, then label them accordingly in the `busiest_days` column.

In [12]:
# Get order frequency by day
day_counts = df['order_dow'].value_counts().sort_values(ascending=False)

In [13]:
# Identify busiest and slowest 2 days
busiest_days = day_counts.head(2).index
slowest_days = day_counts.tail(2).index

In [14]:
# Map values
df['busiest_days'] = df['order_dow'].apply(
    lambda x: 'Busiest days' if x in busiest_days 
    else ('Slowest days' if x in slowest_days else 'Regular days')
)

In [15]:
# Check result
df['busiest_days'].value_counts()

busiest_days
Regular days    12916111
Busiest days    11864412
Slowest days     7624336
Name: count, dtype: int64

### Analysis: Busiest Days

- According to the value counts of `order_dow`,
- The two busiest days of the week are:
  - Saturday (0) and Sunday (1)
- The two slowest days are:
  - Wednesday (3) and Thursday (4)
- All other days are categorized as "Regular days" in the `busiest_days` column.

## Step 4: Create a column for busiest periods of the day

Categorize the order hour (`order_hour_of_day`) into:
- Most orders (top 4 busiest hours)
- Average orders
- Fewest orders (bottom 4 slowest hours)

In [16]:
# Count orders per hour
hour_counts = df['order_hour_of_day'].value_counts().sort_values(ascending=False)

In [17]:
# Define thresholds
most_busy = hour_counts.head(4).index
least_busy = hour_counts.tail(4).index

### Order Counts by Day of Week and Hour of Day

The following tables show the number of orders placed on each day of the week and each hour of the day.
This helps us identify the busiest and slowest time periods.

In [18]:
# Create a table showing number of orders per hour
hour_table = df['order_hour_of_day'].value_counts().sort_index().rename_axis('Hour of Day').reset_index(name='Order Count')
hour_table

,Hour of Day,Order Count
0,0,218769
1,1,115700
2,2,69375
3,3,51281
4,4,53242
5,5,87961
6,6,290493
7,7,891054
8,8,1718118
9,9,2454203


### Analysis: Busiest Periods of the Day

- Based on the `order_hour_of_day` frequency:
  - **Most orders** occur at: 10 AM, 11 AM, 2 PM, and 3 PM
  - **Fewest orders** occur during early hours: 3 AM, 4 AM, 5 AM, and 6 AM
  - All other hours are labeled as "Average orders"

In [19]:
# Create column
df['busiest_period_of_day'] = df['order_hour_of_day'].apply(
    lambda x: 'Most orders' if x in most_busy 
    else ('Fewest orders' if x in least_busy else 'Average orders')
)

In [20]:
# Check
df['busiest_period_of_day'].value_counts()

busiest_period_of_day
Average orders    21293820
Most orders       10849180
Fewest orders       261859
Name: count, dtype: int64

### Analysis: Busiest Days

Based on the value counts from the `order_dow` column, the following mapping was used to classify the two busiest and two slowest days:

| Day of Week    | order_dow | Category        |
|----------------|------------|-----------------|
| Saturday       | 0          | Busiest days    |
| Sunday         | 1          | Busiest days    |
| Wednesday      | 3          | Slowest days    |
| Thursday       | 4          | Slowest days    |
| All other days | 2, 5, 6     | Regular days    |

- The column `busiest_days` was created using these groupings.
- This adjustment meets the client’s request to categorize both the two **busiest** and **slowest** days of the week.

In [21]:
df['busiest_days'] = df['order_dow'].apply(
    lambda x: 'Busiest days' if x in busiest_days 
    else ('Slowest days' if x in slowest_days else 'Regular days')
)

In [22]:
# Create a table showing number of orders per day of week

dow_table = df['order_dow'].value_counts().sort_index().rename_axis('Day of Week').reset_index(name='Order Count')
dow_table

,Day of Week,Order Count
0,0,6204182
1,1,5660230
2,2,4213830
3,3,3840534
4,4,3783802
5,5,4205791
6,6,4496490


### Analysis: Busiest Days

- According to the value counts of `order_dow`, the two busiest days of the week are:
  - **Saturday (0)** and **Sunday (1)**
- The two slowest days are:
  - **Wednesday (3)** and **Thursday (4)**
- All other days are categorized as "Regular days" in the `busiest_days` column.

## Step 5: Export the modified dataframe

Exporting the new dataframe with all the derived columns to the Prepared Data folder for future analysis.

In [23]:
# Export with new columns
df.to_pickle(os.path.join(path, 'Prepared Data', 'ords_prods_derived.pkl'))

## Task Complete

- New columns created: `price_label`, `busiest_days`, and `busiest_period_of_day`
- Labels verified with `.value_counts()`
- Data exported successfully as `ords_prods_derived.pkl`